# Rivers of the world

This notebook pulls river data from OpenStreetMap (OSM) and plots them in Bokeh plots. The idea is to produce something interactive and fun to play with. 

In the future I might try something like [this](https://www.smithsonianmag.com/arts-culture/these-beautiful-maps-capture-rivers-that-pulse-through-our-world-180971789/)

Perhaps to avoid overloading the OverpassAPI could run it locally: https://github.com/mediasuitenz/docker-overpass-api

## Setup

Import libraries and standard tools used

In [ ]:
import os
import requests
import json

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.palettes import viridis, Category20

from pyproj import Transformer

output_notebook()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
area_UK = "(49.325, -11.25, 59.356, 1.45)"

In [ ]:
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

In [ ]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")
 

In [ ]:
# Place to save data so that the number of OSM requests are reduced when re-loading the same data
path_save = r"data/rivers/"

if not os.path.exists(path_save):
    os.makedirs(path_save)
    

In [ ]:
def get_rivers(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "waterway"
    tag = "river"
    recursion = ""
    output = "geom"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output="geom",
                                    recursion="",
                                    element="way")
    
    return data

In [ ]:
def get_data(area,name):
    file_name = path_save+name+"_data.pkl"

    if os.path.isfile(file_name):
        df_metro = pd.read_pickle(file_name)
    else:
        df_metro = get_rivers(area)
        df_metro.to_pickle(file_name)
        
    return df_metro


In [ ]:
def plot_rivers(data,line_color='color'):

    p = figure(plot_width=800, plot_height=800,
              x_axis_type="mercator", y_axis_type="mercator",
              match_aspect=True)

    #p.add_tile(MAP_TILES['ESRI'])

    tooltips = [("id", "@id"),
                ("name", "@name"),
                ("line", "@line"),
                ("network", "@network"),
                ("operator", "@operator"),
                ("(lat,lon)", "@coordinates")]

    xx = list()
    yy = list()
    names = list()
    colors = list()
    for cnt,way in data.iterrows():


        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(way['geometry'])['lat']),
                                                 list(pd.DataFrame.from_records(way['geometry'])['lon']))

        xx.append(xxx)
        yy.append(yyy)
        names.append(way['name'])
        colors.append(way['color'])


    source = ColumnDataSource({'xs':xx,'ys':yy,'color':colors})

    p.multi_line('xs',
                 'ys',
                 source=source,
                 line_width=2,
                 line_color='color',
          )

    p.xaxis.visible = False
    p.xgrid.visible = False
    
    p.yaxis.visible = False
    p.ygrid.visible = False
            
    return p

## UK red rivers

In [ ]:
data = get_data(area_UK,"UK")

In [ ]:
data['color'] = 'red'

In [ ]:
uk = plot_rivers(data)

In [ ]:
uk.background_fill_color = "black"

In [ ]:
show(uk)

## UK rivers coloured by name

In [ ]:
data = data.dropna(subset=['name'])

names = set(data['name'].dropna())

color_palette = list()
for i in range(len(names)):
    color_palette.append(Category20[20][i%20])

color_mapping = dict(zip(names,color_palette))

data["color"] = data['name'].map(color_mapping)


In [ ]:
uk_colors = plot_rivers(data)

In [ ]:
show(uk_colors)

## All together now

In [ ]:
#output_file("world_rivers.html")

# make a grid
#grid = gridplot([[uk, uk, uk], [uk, uk, uk]], plot_width=250, plot_height=250)

#show(grid)